In [1]:
import requests
import json 
import pandas as pd
from api_keys import api_key
import pprint
import os
import csv
import numpy as np
from census import Census

from sqlalchemy import create_engine

from requests.api import head



### Demographic Characteristics Estimates by Age Groups US Data

In [2]:

# Get url for States by population, Race,age and Sex
url=f"https://api.census.gov/data/2015/pep/charagegroups?get=POP,HISP,RACE,SEX&DATE_=3,4,5,6,7,8&for=state:*&key={api_key}"

#Extracting data from website.
headers={
    "Accept":"application/json",
    "Content-type":"application/json" 
}
#response= requests.request("GET",url, headers=headers, data={})
response= requests.request("GET",url, headers=headers)
myjson=response.json()


In [3]:
data_csv = pd.DataFrame(myjson)
data_csv.to_csv('census_data.csv')

#### Clean DataFrame

In [4]:
column_df={0: "Population",
                                     1:"Hispanic",
                                     2:"Race",
                                     3:"Sex",
                                     4:"Year",
                                     5:"State"}
                                     
data_csv.rename(columns=column_df, inplace=True)

In [5]:
# Replacing Column Values to String
data_csv["Sex"].replace(['0','1','2'],["Both sexes","male","female"], inplace=True)
data_csv["Race"].replace(['0','1','2','3','4','5','6','7','8','9','10','11'],['All races','White alone','Black alone','American Indian and Alaska Native alone',\
    'Asian alone','Native Hawaiian and Other Pacific Islander alone','Two or more races',\
       'White or in combination','Black alone or in combination','American Indian and Alaska Native alone or in combination',\
        'Asian alone or in combination','Native Hawaiian and Other Pacific Islander alone or in combination'], inplace=True)
data_csv["Hispanic"].replace(['0','1','2'],["Both Hispanic Origins","Non-Hispanic","Hispanic"], inplace=True)
data_csv["Year"].replace(['3','4','5','6','7','8'],["2010","2011","2012","2013","2014","2015"], inplace=True)

data_csv["State"].replace(['01','02','04','05','06','08','09','10','11','12','13','15', '16', '17', '18', '19',\
    '20', '21','22', '23', '24', '25', '26', '27', '28', '29', '30', '31','32', '33', '34', '35', '36', '37', '38', '39',\
    '40', '41', '42','44', '45', '46', '47','48','49','50','51','53','54','55','56','72'],\
    ['Alabama','Alaska','Arizona','Arkansas','California','Colorado','Connecticut','Delaware','District of Columbia','Florida',\
    'Georgia','Hawaii', 'Idaho', 'Illinois', 'Indiana', 'Iowa', 'Kansas', 'Kentucky','Louisiana', 'Maine',\
    'Maryland', 'Massachusetts', 'Michigan', 'Minnesota', 'Mississippi', 'Missouri', 'Montana', 'Nebraska',\
    'Nevada', 'New Hampshire', 'New Jersey', 'New Mexico', 'New York', 'North Carolina', 'North Dakota', 'Ohio','Oklahoma',\
    'Oregon', '	Pennsylvania','Rhode Island', 'South Carolina', '	South Dakota', 'Tennessee','Texas','Utah','	Vermont',\
    'Virginia','Washington','West Virginia','Wisconsin','Wyoming','Puerto Rico'], inplace=True)
    

# Droppin some rows from the data
data_csv.drop(data_csv[(data_csv['State']=='Puerto Rico')].index, inplace=True)

# Dropping null values
data_csv=data_csv.dropna()


data_csv

,Population,Hispanic,Race,Sex,Year,State
0,POP,HISP,RACE,SEX,DATE_,state
1,68799,Non-Hispanic,Asian alone or in combination,Both sexes,2011,Alabama
2,2878,Hispanic,Asian alone or in combination,Both sexes,2011,Alabama
3,31609,Both Hispanic Origins,American Indian and Alaska Native alone or in ...,male,2010,Alabama
4,25955,Non-Hispanic,American Indian and Alaska Native alone or in ...,male,2010,Alabama
...,...,...,...,...,...,...
33044,117,Hispanic,Native Hawaiian and Other Pacific Islander alo...,female,2012,Wyoming
33045,484843,Non-Hispanic,White alone,Both sexes,2010,Wyoming
33046,44943,Hispanic,White alone,Both sexes,2010,Wyoming
33047,1159,Both Hispanic Origins,Native Hawaiian and Other Pacific Islander alo...,Both sexes,2010,Wyoming


In [6]:
# Renamed clean Census data
census_data=data_csv.iloc[1: , :]
census_data

,Population,Hispanic,Race,Sex,Year,State
1,68799,Non-Hispanic,Asian alone or in combination,Both sexes,2011,Alabama
2,2878,Hispanic,Asian alone or in combination,Both sexes,2011,Alabama
3,31609,Both Hispanic Origins,American Indian and Alaska Native alone or in ...,male,2010,Alabama
4,25955,Non-Hispanic,American Indian and Alaska Native alone or in ...,male,2010,Alabama
5,1252,Hispanic,Native Hawaiian and Other Pacific Islander alone,female,2010,Alabama
...,...,...,...,...,...,...
33044,117,Hispanic,Native Hawaiian and Other Pacific Islander alo...,female,2012,Wyoming
33045,484843,Non-Hispanic,White alone,Both sexes,2010,Wyoming
33046,44943,Hispanic,White alone,Both sexes,2010,Wyoming
33047,1159,Both Hispanic Origins,Native Hawaiian and Other Pacific Islander alo...,Both sexes,2010,Wyoming


In [7]:
# Coverting column "Population" to float
census_data['Population']=census_data["Population"].astype(float)
census_data

c:\Users\madin\anaconda3\envs\PythonDataOne\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,Population,Hispanic,Race,Sex,Year,State
1,68799.0,Non-Hispanic,Asian alone or in combination,Both sexes,2011,Alabama
2,2878.0,Hispanic,Asian alone or in combination,Both sexes,2011,Alabama
3,31609.0,Both Hispanic Origins,American Indian and Alaska Native alone or in ...,male,2010,Alabama
4,25955.0,Non-Hispanic,American Indian and Alaska Native alone or in ...,male,2010,Alabama
5,1252.0,Hispanic,Native Hawaiian and Other Pacific Islander alone,female,2010,Alabama
...,...,...,...,...,...,...
33044,117.0,Hispanic,Native Hawaiian and Other Pacific Islander alo...,female,2012,Wyoming
33045,484843.0,Non-Hispanic,White alone,Both sexes,2010,Wyoming
33046,44943.0,Hispanic,White alone,Both sexes,2010,Wyoming
33047,1159.0,Both Hispanic Origins,Native Hawaiian and Other Pacific Islander alo...,Both sexes,2010,Wyoming


In [8]:
# Checking missing values. 
census_data.isnull().sum()

Population    0
Hispanic      0
Race          0
Sex           0
Year          0
State         0
dtype: int64

In [9]:
census_data=census_data.rename(columns={'Population': 'population', 'Hispanic': 'hispanic', "Race":'race', "Sex":'sex', 'Year':'year',"State":'state'})
census_data

,population,hispanic,race,sex,year,state
1,68799.0,Non-Hispanic,Asian alone or in combination,Both sexes,2011,Alabama
2,2878.0,Hispanic,Asian alone or in combination,Both sexes,2011,Alabama
3,31609.0,Both Hispanic Origins,American Indian and Alaska Native alone or in ...,male,2010,Alabama
4,25955.0,Non-Hispanic,American Indian and Alaska Native alone or in ...,male,2010,Alabama
5,1252.0,Hispanic,Native Hawaiian and Other Pacific Islander alone,female,2010,Alabama
...,...,...,...,...,...,...
33044,117.0,Hispanic,Native Hawaiian and Other Pacific Islander alo...,female,2012,Wyoming
33045,484843.0,Non-Hispanic,White alone,Both sexes,2010,Wyoming
33046,44943.0,Hispanic,White alone,Both sexes,2010,Wyoming
33047,1159.0,Both Hispanic Origins,Native Hawaiian and Other Pacific Islander alo...,Both sexes,2010,Wyoming


### Store CSV into DataFrame

In [10]:
csv_file = 'Resources/NCHS_Causes_of_Death_2005-2015.csv'
Causes_of_death = pd.read_csv(csv_file)
Causes_of_death


,Year,Cause of Death,State,State FIPS Code,HHS Region,Age Range,Benchmark,Locality,Observed Deaths,Population,Expected Deaths,Potentially Excess Deaths,Percent Potentially Excess Deaths
0,2005,Cancer,Alabama,AL,4,0-49,2005 Fixed,All,756.0,3148377.0,451.0,305.0,40.3
1,2005,Cancer,Alabama,AL,4,0-49,2005 Fixed,Metropolitan,556.0,2379871.0,341.0,217.0,39.0
2,2005,Cancer,Alabama,AL,4,0-49,2005 Fixed,Nonmetropolitan,200.0,768506.0,111.0,89.0,44.5
3,2005,Cancer,Alabama,AL,4,0-49,2010 Fixed,All,756.0,3148377.0,421.0,335.0,44.3
4,2005,Cancer,Alabama,AL,4,0-49,2010 Fixed,Metropolitan,556.0,2379871.0,318.0,238.0,42.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...
205915,2015,Unintentional Injury,Wyoming,WY,8,0-84,2010 Fixed,Metropolitan,93.0,175787.0,36.0,57.0,61.3
205916,2015,Unintentional Injury,Wyoming,WY,8,0-84,2010 Fixed,Nonmetropolitan,259.0,400056.0,82.0,177.0,68.3
205917,2015,Unintentional Injury,Wyoming,WY,8,0-84,Floating,All,352.0,575843.0,137.0,215.0,61.1
205918,2015,Unintentional Injury,Wyoming,WY,8,0-84,Floating,Metropolitan,93.0,175787.0,42.0,51.0,54.8


In [11]:
# Clean Causes of Death DataFrame. State was removed from the data since we will have a duplicate on joining two data frames.
new_causes_of_death = Causes_of_death[['Year', 'Cause of Death','Benchmark', 'Locality', 'Age Range',  'Observed Deaths', 'Expected Deaths']].copy()
new_causes_of_death.head()
 
# Dropping null values
new_causes_of_death=new_causes_of_death.dropna()



In [12]:
# Nonmetropolitan does not work
ncod_year=new_causes_of_death.loc[new_causes_of_death["Year"]>=2010]
ncod_benchmark = new_causes_of_death.loc[new_causes_of_death['Benchmark'] =='2010 Fixed']
ncod_age = new_causes_of_death.loc[new_causes_of_death['Age Range'] =='0-84']
ncod_locality=new_causes_of_death.loc[Causes_of_death['Locality']!='All']





ncod1_year=new_causes_of_death.loc[new_causes_of_death["Year"]>=2010]
ncod1_benchmark = ncod1_year.loc[ncod1_year['Benchmark'] =='2010 Fixed']
ncod1_age = ncod1_benchmark.loc[ncod1_benchmark['Age Range'] =='0-84']
ncod1_locality=ncod1_age.loc[ncod1_age['Locality']!='All']
ncod_full=ncod1_locality
ncod_full

,Year,Cause of Death,Benchmark,Locality,Age Range,Observed Deaths,Expected Deaths
93685,2010,Cancer,2010 Fixed,Metropolitan,0-84,6415.0,4589.0
93686,2010,Cancer,2010 Fixed,Nonmetropolitan,0-84,2464.0,1690.0
93756,2010,Cancer,2010 Fixed,Metropolitan,0-84,492.0,423.0
93757,2010,Cancer,2010 Fixed,Nonmetropolitan,0-84,306.0,240.0
93827,2010,Cancer,2010 Fixed,Metropolitan,0-84,8409.0,7689.0
...,...,...,...,...,...,...,...
205772,2015,Unintentional Injury,2010 Fixed,Nonmetropolitan,0-84,516.0,155.0
205843,2015,Unintentional Injury,2010 Fixed,Metropolitan,0-84,1670.0,869.0
205844,2015,Unintentional Injury,2010 Fixed,Nonmetropolitan,0-84,646.0,325.0
205915,2015,Unintentional Injury,2010 Fixed,Metropolitan,0-84,93.0,36.0


In [13]:
ncod_full=ncod_full.rename(columns={'Year' : 'year', 'Cause of Death': 'cause_of_death', 'Age Range': 'age_range',\
     'Benchmark': 'benchmark', 'Locality': 'locality', 'Observed Deaths': 'observed_deaths', 'Expected Deaths': 'expected_deaths' })

ncod_full



,year,cause_of_death,benchmark,locality,age_range,observed_deaths,expected_deaths
93685,2010,Cancer,2010 Fixed,Metropolitan,0-84,6415.0,4589.0
93686,2010,Cancer,2010 Fixed,Nonmetropolitan,0-84,2464.0,1690.0
93756,2010,Cancer,2010 Fixed,Metropolitan,0-84,492.0,423.0
93757,2010,Cancer,2010 Fixed,Nonmetropolitan,0-84,306.0,240.0
93827,2010,Cancer,2010 Fixed,Metropolitan,0-84,8409.0,7689.0
...,...,...,...,...,...,...,...
205772,2015,Unintentional Injury,2010 Fixed,Nonmetropolitan,0-84,516.0,155.0
205843,2015,Unintentional Injury,2010 Fixed,Metropolitan,0-84,1670.0,869.0
205844,2015,Unintentional Injury,2010 Fixed,Nonmetropolitan,0-84,646.0,325.0
205915,2015,Unintentional Injury,2010 Fixed,Metropolitan,0-84,93.0,36.0


In [14]:
# Coverting column "Year" to an integer to be able to join two DataFrames on this column.
census_data['year']=census_data["year"].astype(int)

#### Join two DataFrames

In [15]:
final_df = pd.merge(census_data, ncod_full, how='inner', on = 'year')
final_df

,population,hispanic,race,sex,year,state,cause_of_death,benchmark,locality,age_range,observed_deaths,expected_deaths
0,68799.0,Non-Hispanic,Asian alone or in combination,Both sexes,2011,Alabama,Cancer,2010 Fixed,Metropolitan,0-84,6445.0,4690.0
1,68799.0,Non-Hispanic,Asian alone or in combination,Both sexes,2011,Alabama,Cancer,2010 Fixed,Nonmetropolitan,0-84,2547.0,1713.0
2,68799.0,Non-Hispanic,Asian alone or in combination,Both sexes,2011,Alabama,Cancer,2010 Fixed,Metropolitan,0-84,513.0,442.0
3,68799.0,Non-Hispanic,Asian alone or in combination,Both sexes,2011,Alabama,Cancer,2010 Fixed,Nonmetropolitan,0-84,323.0,250.0
4,68799.0,Non-Hispanic,Asian alone or in combination,Both sexes,2011,Alabama,Cancer,2010 Fixed,Metropolitan,0-84,8476.0,7972.0
...,...,...,...,...,...,...,...,...,...,...,...,...
16523995,2350.0,Hispanic,American Indian and Alaska Native alone or in ...,male,2014,Wyoming,Unintentional Injury,2010 Fixed,Nonmetropolitan,0-84,465.0,156.0
16523996,2350.0,Hispanic,American Indian and Alaska Native alone or in ...,male,2014,Wyoming,Unintentional Injury,2010 Fixed,Metropolitan,0-84,1621.0,863.0
16523997,2350.0,Hispanic,American Indian and Alaska Native alone or in ...,male,2014,Wyoming,Unintentional Injury,2010 Fixed,Nonmetropolitan,0-84,639.0,325.0
16523998,2350.0,Hispanic,American Indian and Alaska Native alone or in ...,male,2014,Wyoming,Unintentional Injury,2010 Fixed,Metropolitan,0-84,88.0,36.0


In [37]:
# Finding the statistical description. 
final_df.describe()

,population,year,observed_deaths,expected_deaths
count,1.652400e+07,1.652400e+07,1.652400e+07,1.652400e+07
mean,6.930715e+05,2.012500e+03,4.581750e+03,3.404628e+03
std,2.005353e+06,1.707825e+00,2.351351e+04,1.891336e+04
min,1.500000e+01,2.010000e+03,1.300000e+01,1.900000e+01
25%,6.314250e+03,2.011000e+03,3.510000e+02,2.190000e+02
50%,3.727650e+04,2.012500e+03,9.175000e+02,6.020000e+02
75%,3.383580e+05,2.014000e+03,2.542000e+03,1.644750e+03
max,3.914482e+07,2.015000e+03,4.003560e+05,3.599100e+05


### Connect to local database

In [30]:
protocol = 'postgresql'
username = 'postgres'
password = 'M@dina1993'
host = 'localhost'
port = 5432
database_name = 'death_db'
rds_connection_string = f'{protocol}://{username}:{password}@{host}:{port}/{database_name}'
engine = create_engine(rds_connection_string)

### Check for tables

In [31]:
engine.table_names()

[]

In [32]:
census_data.columns

Index(['population', 'hispanic', 'race', 'sex', 'year', 'state'], dtype='object')

In [33]:
ncod_full['benchmark'].value_counts()

2010 Fixed    3000
Name: benchmark, dtype: int64

### Use Pandas to load CSV converted DataFrame into database

In [34]:
census_data.to_sql('census_db', con=engine, if_exists="append", index=False, chunksize=1000)

In [35]:
ncod_full.to_sql('death_causes', con=engine, if_exists="append", index=False)

#### Confirm data has been added by querying the customer_name table

In [ ]:
pd.read_sql_query('select * from census_db', con=engine).head()

In [ ]:
pd.read_sql_query('select * death_causes', con=engine).head()